# Neptune API tour


## Introduction

This guide will show you how to:

* Install neptune-client
* Connect Neptune to your script and create the first run
* Log simple metrics to Neptune and explore the in the UI
* Log learning curves, images and model binaries from Keras training and see those in the Neptune app
* Fetch the data you logged to Neptune directly into your notebook and analyze them 

By the end of it, you will execute your first run and see it in Neptune!

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

* If you are running the notebook on your local machine, you need to have [Python](https://www.python.org/downloads/) and [pip](https://pypi.org/project/pip/) installed.
* If you want to see the example recorded to your own workspace instead:
    * Create a Neptune account → [Take me to registration](https://neptune.ai/register)
    * Create a Neptune project that you will use for tracking metadata → [Tell me more about projects](https://docs.neptune.ai/administration/projects)

## Install Neptune and dependencies

In [ ]:
! pip install neptune-client

## Initialize Neptune

Connect your script to Neptune application and create new run.

In [ ]:
import neptune.new as neptune

run = neptune.init(project="common/quickstarts", api_token="ANONYMOUS")

You tell Neptune: 

* **who you are**: your Neptune API token `api_token` 
* **where you want to send your data**: your Neptune `project`.

At this point you have new run in Neptune. For now on you will use `run` to log metadata to it.

---

**Note**


Instead of logging data to the public project 'common/quickstarts' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your [Neptune API token](https://docs.neptune.ai/getting-started/installation#authentication-neptune-api-token)
2. Pass the token to ``api_token`` argument of ``neptune.init()`` method: ``api_token=YOUR_API_TOKEN``
3. Pass your project to the ``project`` argument of ``neptune.init()``.

For example:

```python
neptune.init(project="YOUR_WORKSPACE/YOUR_PROJECT", 
             api_token="YOUR_API_TOKEN")
```

## Basic Example

Lets start with something super simple.

You will:
* log parameters
* log a training and validation scores
* log loss
* add name to the run
* append two tags

In [ ]:
params = {"learning_rate": 0.1}

# log params
run["parameters"] = params

# log name and append tags
run["sys/name"] = "basic-colab-example"
run["sys/tags"].add(["colab", "intro"])

# log loss during training
for epoch in range(100):
    run["train/loss"].log(0.99**epoch)

# log train and validation scores
run["train/accuracy"] = 0.95
run["valid/accuracy"] = 0.93

Runs can be viewed as dictionary-like structures - **namespaces** - that you can define in your code. You can apply hierarchical structure to your metadata that will be reflected in the UI as well. Thanks to this you can easily organize your metadata in a way you feel is most convenient.

There is one special namespace: **system namespace**, denoted `sys`. You can use it to add name and tags to the run.

Try to change the values and re-execute to see your runs appear in the dashboard.

All `basic-colab-example` runs are grouped in [this dashboard view](https://app.neptune.ai/o/common/org/quickstarts/experiments?split=tbl&dash=charts&viewId=96fe9215-15ff-4bf8-9096-9d06a99a8b36).

![colab-runs](https://user-images.githubusercontent.com/41324509/184083881-90568b31-166b-4fef-9887-8cec587de1ae.png)

There are many other things that you can log to neptune:

* Images and charts
* Artifacts like model weights or results
* Text values
* Hardware consumption
* Code snapshots
* and more

You can go and see all that in the [documentation](https://docs.neptune.ai/user-guides/logging-and-managing-runs-results/logging-runs-data), and also check out the next example to see some of those.


**Stop logging**  

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

## Keras classification example [Advanced]

Install and import your machine learning libraries

In [ ]:
! pip install scikit-plot==0.3.7 tensorflow==2.4.1

Get the data

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Prepare the model

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

To log metrics after every batch and epoch let's create `NeptuneLogger` callback:

In [ ]:
from tensorflow.keras.callbacks import Callback


class NeptuneLogger(Callback):
    def on_batch_end(self, batch, logs=None):
        if logs is None:
            logs = {}
        for log_name, log_value in logs.items():
            run[f"batch/{log_name}"].log(log_value)

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        for log_name, log_value in logs.items():
            run[f"epoch/{log_name}"].log(log_value)

Now we simply need to create a run.
I will tag it with the name `advanced` and log hyperparameters `epoch_nr` and `batch_size`: 

In [ ]:
EPOCH_NR = 5
BATCH_SIZE = 32

run = neptune.init(project="common/quickstarts", api_token="ANONYMOUS")

# log params
run["parameters/epoch_nr"] = EPOCH_NR
run["parameters/batch_size"] = BATCH_SIZE

# log name and append tag
run["sys/name"] = "keras-metrics"
run["sys/tags"].add("advanced")

Now we pass our `NeptuneLogger` as keras callback and thats it.

In [ ]:
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=EPOCH_NR,
    batch_size=BATCH_SIZE,
    validation_data=(x_test, y_test),
    callbacks=[NeptuneLogger()],
)

You can click on the run link above and monitor your learning curves as it is training!

![advanced](https://user-images.githubusercontent.com/41324509/184084517-a1d654c3-e9e8-4f54-823d-cfb921d7cee2.png)


Great thing is, you can log more things if you need to during or after the training is finished.

For example, let's calculate some additional metrics on test data and log them.

In [ ]:
import numpy as np

y_test_pred = np.asarray(model.predict(x_test))
y_test_pred_class = np.argmax(y_test_pred, axis=1)

In [ ]:
from sklearn.metrics import f1_score

run["test/f1"] = f1_score(y_test, y_test_pred_class, average="micro")

We can log diagnostic charts like confusion matrix or ROC AUC curve.

In [ ]:
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_confusion_matrix, plot_roc

fig, ax = plt.subplots(figsize=(16, 12))
plot_confusion_matrix(y_test, y_test_pred_class, ax=ax)
run["diagnostic_charts"].log(neptune.types.File.as_image(fig))

fig, ax = plt.subplots(figsize=(16, 12))
plot_roc(y_test, y_test_pred, ax=ax)
run["diagnostic_charts"].log(neptune.types.File.as_image(fig))

![Saved charts](https://user-images.githubusercontent.com/41324509/184084858-8260adba-b611-4ecd-8a30-ccdd6d0a40e5.png)

We can also log model weights to Neptune.

In [ ]:
model.save("my_model.h5")
run["model"].upload("my_model.h5")

![Model](https://user-images.githubusercontent.com/41324509/184085565-470e1c85-a4f2-4992-90b8-43e1d1a10724.png)

With that you can share models with your teammates easily.

You can play around and execute this run with different parameters and see results and compare them.

Like I've done here:

![alt text](https://neptune.ai/wp-content/uploads/docs-api-tour-compare.png)

A cool thing is, once things are logged to Neptune you can access them from wherever you want.
Let me show you. 



## Access data you logged programatically 

## Getting the project's leaderboard

You can use the project object to get aggregate data of all your runs. Here, fetch only runs with tag 'advanced'.

In [ ]:
my_project = neptune.get_project(name="common/quickstarts", api_token="ANONYMOUS")
run_df = my_project.fetch_runs_table(tag=["advanced"]).to_pandas()
run_df.head()

**Stop logging** 

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

## Getting the run's metadata

Neptune lets you fetch whatever you logged to it directly to your notebooks and scripts.

In [ ]:
run = neptune.init(project="common/quickstarts", api_token="ANONYMOUS", run="QUI-80989")

Now that your run is *fetched* you can download the metadata.

In [ ]:
batch_size = run["parameters/batch_size"].fetch()
last_batch_acc = run["batch/accuracy"].fetch_last()
print(f"batch_size: {batch_size}")
print(f"last_batch_acc: {last_batch_acc}")

You can even download artifacts from that run if you want to:



In [ ]:
run["model"].download()

In [ ]:
! ls ./

**Stop logging**  

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
run.stop()

## Learn more about Neptune

Read about other Neptune features, create your free account and start logging!

[Go to Neptune](https://neptune.ai/register)